# The Game 'Angry Birds'
## Author: Hasan Sayginel

## Introduction

**Angry Birds** is a world-famous series of games, popular among smartphone and tablet users. Due to its huge success, new versions of the game has been and are still being released. However, each game has the same outline, where the users launch birds from the ground at an angle in order to topple and kill green 'evil' pigs standing on blocks to save their eggs. The game combines different concepts from the study of mechanics and in this notebook, we will consider projectile motion, momentum, collisions and torques in order to code our own version of the game. [1]

In [1]:
## importing modules ##
import numpy as np
from vpython import sphere, color, rate, canvas, vector, curve, label, box, mag, cross, random

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Setting up the background

The ground is represented by the horizontal green line in the canvas. For the purposes of calculation, we will work in units where one screen unit is equal to 1 metre. Our ground is 16 m long. The code below sets the canvas, places the bird at the start and the target at a random position between 5 m to 15 m from the start. 

**Bird**

A yellow ball is used to represent our bird which is initially placed at the start. 

**Target**

In our version of the game, we will not have green pigs, but we will have a rectangular target, 2 m long and 0.5 m wide. The target is given an arbitrary mass, which can be changed. Increasing the mass of the target, effectively increases the difficulty of the game as it becomes more difficult to topple the target. The reasoning behind this is explained later when the torque equations are introduced. 

In [2]:
scene = canvas(width=640, height=480, center=vector(8,0,0),range=8)
ground = curve(pos=[vector(0,0,0),vector(16,0,0)],color = color.green)

## setting up target with random x position ##
x = np.random.randint(5,15) # x-position of the target
target = box(pos=vector(x,1,0), width=0.5, height=2, color = color.blue) # rectangular target
m_target = 100 # mass of the target

## properties of the bird ##
m = 0.1 # kg, mass of the bird
radius_bird = 0.05 # radius of the bird
bird = sphere(pos = vector(0,0,0),radius=0.3,make_trail=True,color=color.yellow) # draw bird as a white sphere

<IPython.core.display.Javascript object>

## Outline and Physics of the Game
                                                                                   
### Projectile Motion 

The user sets the bird into motion by inputting two parameters - the initial speed and launch angle in degrees. The bird follows a parabolic path and either hits or misses the target. In our version, this motion is modelled using equations for projectile motion:

***Equation 1:***

$$ x(t) = x_0 + v_0 t \cos \theta $$                   

***Equation 2:***

$$ y(t) = y_0 + v_0 t \sin \theta -\frac{1}{2}g t^2 $$        

Motion in horizontal and vertical directions are independent, thus the above equations can be used to find the new x and y positions of the bird, update it with time and trace the path by using vpython. Bird stops if it hits the ground before it reaches the target or if it misses the target. 

### Collision

A variable named **hit_tolerance** is defined which is the sum of the radius of the bird and the distance from the centre of mass of the target to its edge. The variable is used as a condition to decide if the bird hits the target. If the distance between the x-position of target and the position vector of the bird is less than the hit_tolerance, a collision is said to have occured. If a collision does not occur, the user is asked to try again by entering different initial parameters. [2] [3]

***If the bird hits the target:***

Momentum of the bird, the force, thus the torque it applies on the target are considered to determine if the target will topple. 



### Momentum of the bird 

\- Horizontal component:
 
***Equation 3:***
$$ p_x = mv_0\cos(\theta) $$                    
 
\- Vertical component:

***Equation 4:***
$$ p_y = mv_0\sin(\theta) - mgt $$                 
                            
The momentum of the bird is expressed in vector form as $\mathbf{p}_{ball} $ and is used to calculate the force applied by the bird:

***Equation 5:***
$$ \mathbf{F}_{applied} = \frac{\mathbf{p}_{ball}}{\Delta{t}} $$

### Torques acting on the target 

Two types of torque with opposite directions act on the target when a collision occurs. 

** Restoring torque **

The following formula is used to calculate the magnitude of the restoring torque directly (full derivation of the formula to be found in reference [3]):

***Equation 6:***
$$ |\tau_{restoring}| = m_{target} g\frac{\omega}{2}$$           

, where $m_{target}$ is the mass of the target. If $m_{target}$ is increased, the magnitude of the restoring torque increases, therefore a greater torque needs to be applied in order to topple the target. The restoring torque is constant for a constant mass.

** Applied torque **

The applied torque vector is calculated using the equation below:

***Equation 7:***
$$ \mathbf{\tau}_{applied} = \mathbf{F}_{applied} \times \mathbf{d}_a $$    

The magnitude of the vector obtained using the above equation gives us the magnitude of the applied torque. 

As the point of rotation of the target is its **lower right-end corner**, the restoring torque acts in **anticlockwise direction** whereas the applied torque acts in **clockwise direction**. If the magnitude of the applied torque is greater than the magnitude of the restoring torque, the target will topple over and the user will win the game. If it is not, then the user will be asked to enter new initial parameters and try again.

In [3]:
## initial parameters ##
x0 = 0.0 # initial x-coordinate
y0 = 0.0 # initial y-coordinate
t = 0.0 # initial time
xt = 0.0 # initialise xt
yt = 0.0 # initialise yt

# function to ask user input for initial speed
def v():
    """Returns initial speed"""
    v0 = float(input("Input the initial speed in meters/second: "))
    return v0

# function to ask user input for launch angle
def theta():
    """Returns launch angle"""
    dtheta = float(input("Input the initial angle in degrees: ")) # 0<angle<90
    theta = np.radians(dtheta)
    return theta

In [ ]:
v0 = v() # initial speed
theta = theta() # launch angle in radians

# other physical parameters
g = 9.81 # m/s^-2, gravitational acceleration 
delta_t = 0.01 # impact time 

l = 0.25 # distance from CM of target to edge of target
hit_tolerance = radius_bird + l # parameter for if loop

t_appliedmag = 1 # initialise, can take any value which is less than t_restoremag
t_restoremag = 2 # initialise, can take any value which is greater than t_appliedmag

lose = label(pos=vector(14,4,0), text = 'TRY AGAIN!') # TRY AGAIN text on canvas for a miss

while t_appliedmag <= t_restoremag: # end loop when applied torque is greater
    while xt <= x - 0.3 and yt >= 0: # animate until bird hits ground or reaches edge of target
        rate(20) # updates in animation per second
        xt = x0 + v0*t*np.cos(theta) # horizontal distance of bird
        yt = y0 + v0*t*np.sin(theta) - 0.5*g*t**2 # vertical distance of bird
        t = t + 0.005 # time (time interval for animation = 0.01 s)
        bird.pos = vector(xt,yt,0) # update position of bird on animation
        
    dist = x - mag(bird.pos) # distance between bird and target
    d_a = vector(-0.5,yt,0) # vector right end corner to the point of collision
    
    if abs(dist) < hit_tolerance and yt <=2.3: # if collision occurs (yt<= 2.3 corrects the error):
        p_x = m*v0*np.cos(theta) # momentum in horizontal direction
        p_y = m*v0*np.sin(theta) - m*g*t # momentum in vertical direction
        p = vector(p_x,p_y,0) # momentum vector
        
        f_applied = p / delta_t # magnitude of force applied on the target by bird
        
        t_restoremag = m_target*g*0.25 # magnitude of the restoring torque
        t_applied = cross(f_applied, d_a) # applied torque vector
        t_appliedmag = mag(t_applied) # magnitude of the applied torque
    
        pmag = mag(p) # momentum
        
        print("The height of the impact point is", yt,"m.")
        print("The bird's momentum at the point of impact is", pmag,"kg ms^-1." )
        print("The magnitude of the torque applied on the target by the bird is",t_appliedmag,"Nm.")
        print("The magnitude of the restoring torque is",t_restoremag,"Nm.")
        
    else:
        lose # TRY AGAIN label
        
    if t_appliedmag <= t_restoremag: # check again and set parameters to zero for the next run
        bird.pos = vector(0,0,0)
        x0 = 0.0
        y0 = 0.0
        xt = 0.0
        yt = 0.0
        t = 0.0
        lose 
        v0 = v() # input new speed
        dtheta = float(input("Input the initial angle in degrees: ")) # input new angle in degrees
        theta = np.radians(dtheta) # angle converted to radians

       
label_win = label(pos=vector(14,4,0), text = 'WIN!') # label message "WIN!"
label_win 
lose.visible = False # TRY AGAIN label disappears
print("WIN!!! Congratulations :)")

## topple target ##
target.up = vector(1,0,0) # change upward direction
target.pos = vector(x,0.5,0) # rest on ground

Input the initial speed in meters/second: 15
Input the initial angle in degrees: 15


## Additional note on code for the game

The animation of the bird is stopped (line 17) when it reaches the edge of the target, regardless of whether it hits the target. This is to ensure the time (t) term used in the following momentum and torque equations corresponds to the time that has elapsed between the launch time and time of collision. 

## Improvements 
While the physics implemented is sufficient for a game, it can still be further improved to represent a physical situation more accurately. As an improvement, the effects of air friction and the final horizontal speed of the bird after the collision may be considered when coding the game. 

### Air friction
The effect of air friction which is a force proportional to the velocity of an object may be included in the code to improve the accuracy. The friction force has the form:

$$ \mathbf{F} = - \beta \mathbf{v} $$

### Final horizontal speed of bird > 0
It is assumed that the bird hits the target and comes to rest horizontally. In that analogy, all the initial momentum of the bird is transferred to the target. However, that is not usually the case in real life. The bird may bounce back leading to a greater momentum acting on the target which may be sufficient to topple it. The following equations would add the effect of this to the game:

$$ m_{bird}u = m_{bird}v_{bird} + m_{target}v_{target} $$

$$ e = \frac{v_{target} - v_{bird}}{u} $$

, where e is the coefficient of restitution. A value for e should be assigned in the code to calculate the final horizontal speed of the bird, thus its momentum.


## References

[1] En.wikipedia.org. (2018). Angry Birds. [online] Available at: https://en.wikipedia.org/wiki/Angry_Birds [Accessed 4 Jan. 2018].

[2] Morin, D. (2012). Introduction to classical mechanics. Cambridge: Cambridge University Press, pp.322-331.

[3] Dash, L. (2018). PHAS 1240: Final Assignment 2017/18. UCL DEPARTMENT OF PHYSICS AND ASTRONOMY, pp.2-4.